In [1]:
import numpy as np
import cv2 as cv
from motrackers import SimpleTracker2
from motrackers.utils import select_caffemodel, select_videofile

### Setup 

#### Choose file paths


In [2]:
video_file = select_videofile('..')
prototxt, weights = select_caffemodel('..')
display(video_file, prototxt, weights)

FileChooser(path='..', filename='', show_hidden='False')

FileChooser(path='..', filename='', show_hidden='False')

FileChooser(path='..', filename='', show_hidden='False')

#### Set hyper parameters and constants

In [3]:
print_tracks = True   # print the tracker output if True

video = video_file.selected

model = {"prototxt": prototxt.selected,
         "weights": weights.selected,
         "object_names": {0: 'background', 
                          1: 'aeroplane', 
                          2: 'bicycle', 
                          3: 'bird',
                          4: 'boat',
                          5: 'bottle',
                          6: 'bus', 
                          7: 'car', 
                          8: 'cat', 
                          9: 'chair',
                          10: 'cow', 
                          11: 'diningtable', 
                          12: 'dog', 
                          13: 'horse',
                          14: 'motorbike', 
                          15: 'person', 
                          16: 'pottedplant',
                          17: 'sheep', 
                          18: 'sofa', 
                          19: 'train',
                          20: 'tvmonitor'},
         "threshold": 0.2,
         "confidence_threshold": 0.2,
         "pixel_std":1/127.5,
         "pixel_mean": 127.5,
         "input_size": (300, 300)
        }

max_object_lost_count = 5   # maximum number of object losts counted when the object is being tracked

np.random.seed(12345)
bbox_colors = {key: np.random.randint(0, 255, size=(3,)).tolist() for key in model['object_names'].keys()}

### Initialize
* Initialize video
* Initialize network
* Initialize tracker

In [4]:
cap = cv.VideoCapture(video)
net = cv.dnn.readNetFromCaffe(model["prototxt"], model["weights"])
tracker = SimpleTracker2(max_lost=max_object_lost_count)

In [5]:
(H, W) = (None, None)
writer = None

while True:
    ok, image = cap.read()
    
    if not ok:
        print("Cannot read the video feed.")
        break
    
    if W is None or H is None: 
        (H, W) = image.shape[:2]
    
    image_resized = cv.resize(image, model["input_size"])

    blob = cv.dnn.blobFromImage(image_resized, 
                                 model["pixel_std"], 
                                 model["input_size"], 
                                 (model["pixel_mean"], model["pixel_mean"], model["pixel_mean"]), 
                                 False)

    net.setInput(blob)
    detections = net.forward()

    rows = image_resized.shape[0]
    cols = image_resized.shape[1]
    
    boxes, confidences, classIDs, detections_bbox = [], [], [], []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > model['confidence_threshold']:
            class_id = int(detections[0, 0, i, 1])

            # object location 
            left = int(detections[0, 0, i, 3] * cols) 
            top = int(detections[0, 0, i, 4] * rows)
            right = int(detections[0, 0, i, 5] * cols)
            bottom = int(detections[0, 0, i, 6] * rows)
            
            # scaling factor of image
            height_factor = image.shape[0]/float(model["input_size"][0])
            width_factor = image.shape[1]/float(model["input_size"][1])
            
            # scale object detection bounding box to original image
            left = int(width_factor * left) 
            top = int(height_factor * top)
            
            right = int(width_factor * right)
            bottom = int(height_factor * bottom)
            
            width = right - left
            height = bottom - top
            
            boxes.append([left, top, width, height])
            confidences.append(float(confidence))
            classIDs.append(int(class_id))
        
    indices = cv.dnn.NMSBoxes(boxes, confidences, model["confidence_threshold"], model["threshold"])
    
    class_ids_to_track = []
    if len(indices)>0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            
            detections_bbox.append((x, y, x+w, y+h))
            class_ids_to_track.append(classIDs[i])
            
            clr = [int(c) for c in bbox_colors[classIDs[i]]]
            cv.rectangle(image, (x, y), (x+w, y+h), clr, 2)
            
            label = "{}:{:.4f}".format(model["object_names"][classIDs[i]], confidences[i])
            
            (label_width, label_height), baseLine = cv.getTextSize(
                label, cv.FONT_HERSHEY_SIMPLEX, 0.5, 2)
            
            y_label = max(y, label_height)
            
            cv.rectangle(image, 
                         (x, y_label-label_height),
                         (x+label_width, y_label+baseLine), 
                         (255, 255, 255), 
                         cv.FILLED)
            
            cv.putText(image, label, (x, y_label), cv.FONT_HERSHEY_SIMPLEX, 0.5, clr, 2)
        
    tracks = tracker.update(detections_bbox, class_ids_to_track, confidences)
    
    if print_tracks:
        print("Tracker Outputs:\n", tracks, "\n")
    
    for track in tracks:
        track_count, trackid, class_id, centroid, track_bbox, track_info = track
        
        text = "ID {}".format(trackid)
        
        cv.putText(image, 
                   text, 
                   (centroid[0] - 10, centroid[1] - 10),
                   cv.FONT_HERSHEY_SIMPLEX,
                   0.5, 
                   (0, 255, 0), 
                   2)
        
        cv.circle(image, 
                  (centroid[0], centroid[1]), 
                  4, 
                  (0, 255, 0), 
                  -1)
    
    cv.imshow("image", image)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
    if writer is None:
        fourcc = cv.VideoWriter_fourcc(*"MJPG")
        writer = cv.VideoWriter("output.avi", fourcc, 30, (W, H), True)
    writer.write(image)

writer.release()
cap.release()
cv.destroyWindow("image")

Tracker Outputs:
 [(1, 0, 15, array([590, 299]), (541, 126, 640, 473), {'max_score': 0.0, 'lost': 0, 'score': 0.0}), (1, 1, 15, array([315, 246]), (285, 156, 345, 337), {'max_score': 0.0, 'lost': 0, 'score': 0.0}), (1, 2, 15, array([533, 194]), (486, 20, 580, 368), {'max_score': 0.0, 'lost': 0, 'score': 0.0}), (1, 3, 4, array([345,  45]), (217, 6, 473, 84), {'max_score': 0.0, 'lost': 0, 'score': 0.0})] 

Tracker Outputs:
 [(2, 0, 15, array([590, 299]), (541, 126, 640, 473), {'max_score': 0.0, 'lost': 1, 'score': 0.0}), (2, 1, 15, array([312, 248]), array([281, 158, 343, 339]), {'max_score': 0.0, 'lost': 0, 'score': 0.27704158425331116}), (2, 2, 15, array([542, 224]), array([492,  12, 593, 436]), {'max_score': 0.0, 'lost': 0, 'score': 0.7952107191085815}), (2, 3, 4, array([345,  45]), (217, 6, 473, 84), {'max_score': 0.0, 'lost': 1, 'score': 0.0})] 

Tracker Outputs:
 [(3, 0, 15, array([588, 284]), array([539, 105, 637, 464]), {'max_score': 0.0, 'lost': 0, 'score': 0.7579631209373474}),

Tracker Outputs:
 [(24, 0, 15, array([424, 279]), array([358,  88, 490, 470]), {'max_score': 0.0, 'lost': 0, 'score': 0.4059274196624756}), (24, 1, 15, array([293, 229]), array([256, 139, 330, 320]), {'max_score': 0.0, 'lost': 0, 'score': 0.7630714178085327}), (24, 3, 4, array([312,  46]), array([198,   4, 426,  89]), {'max_score': 0.0, 'lost': 2, 'score': 0.8712831735610962}), (24, 4, 16, array([537, 170]), array([482,   4, 593, 337]), {'max_score': 0.0, 'lost': 0, 'score': 0.99678635597229})] 

Tracker Outputs:
 [(25, 0, 15, array([415, 278]), array([345,  81, 486, 475]), {'max_score': 0.0, 'lost': 0, 'score': 0.9948922395706177}), (25, 1, 15, array([295, 232]), array([258, 144, 332, 320]), {'max_score': 0.0, 'lost': 0, 'score': 0.9536147713661194}), (25, 3, 4, array([312,  46]), array([198,   4, 426,  89]), {'max_score': 0.0, 'lost': 3, 'score': 0.8712831735610962}), (25, 4, 16, array([533, 171]), array([477,   4, 590, 339]), {'max_score': 0.0, 'lost': 0, 'score': 0.7284836769104004

Tracker Outputs:
 [(48, 0, 15, array([300, 232]), array([228,  17, 373, 448]), {'max_score': 0.0, 'lost': 0, 'score': 0.9966802597045898}), (48, 4, 16, array([552, 166]), array([484,  11, 620, 321]), {'max_score': 0.0, 'lost': 0, 'score': 0.2854361832141876})] 

Tracker Outputs:
 [(49, 0, 15, array([295, 258]), array([219,  52, 371, 464]), {'max_score': 0.0, 'lost': 0, 'score': 0.9986825585365295}), (49, 4, 16, array([557, 168]), array([499,   6, 616, 331]), {'max_score': 0.0, 'lost': 0, 'score': 0.33141815662384033}), (49, 6, 16, array([ 45, 243]), (4, 203, 87, 283), {'max_score': 0.0, 'lost': 0, 'score': 0.0})] 

Tracker Outputs:
 [(50, 0, 15, array([299, 267]), array([211,  72, 388, 462]), {'max_score': 0.0, 'lost': 0, 'score': 0.9976934790611267}), (50, 4, 16, array([557, 171]), array([494,  11, 620, 332]), {'max_score': 0.0, 'lost': 0, 'score': 0.32177507877349854}), (50, 6, 16, array([ 41, 243]), array([  2, 203,  81, 284]), {'max_score': 0.0, 'lost': 0, 'score': 0.27678808569908

Tracker Outputs:
 [(72, 0, 15, array([198, 261]), array([108,  80, 288, 443]), {'max_score': 0.0, 'lost': 0, 'score': 0.25884202122688293}), (72, 6, 16, array([ 43, 244]), array([  4, 200,  83, 288]), {'max_score': 0.0, 'lost': 0, 'score': 0.9715941548347473}), (72, 8, 15, array([549, 241]), array([501,  56, 597, 427]), {'max_score': 0.0, 'lost': 0, 'score': 0.99882572889328}), (72, 10, 4, array([344,  44]), array([228,   6, 460,  83]), {'max_score': 0.0, 'lost': 0, 'score': 0.4417996108531952}), (72, 11, 15, array([331, 228]), (300, 134, 362, 323), {'max_score': 0.0, 'lost': 0, 'score': 0.0})] 

Tracker Outputs:
 [(73, 0, 15, array([194, 260]), array([106,  81, 283, 440]), {'max_score': 0.0, 'lost': 0, 'score': 0.9982615113258362}), (73, 6, 16, array([ 43, 245]), array([  4, 201,  83, 289]), {'max_score': 0.0, 'lost': 0, 'score': 0.3093146085739136}), (73, 8, 15, array([546, 262]), array([497,  92, 595, 432]), {'max_score': 0.0, 'lost': 0, 'score': 0.9982348680496216}), (73, 10, 4, ar

Tracker Outputs:
 [(87, 0, 15, array([113, 256]), array([ 46,  88, 181, 425]), {'max_score': 0.0, 'lost': 0, 'score': 0.9806451201438904}), (87, 8, 15, array([454, 257]), array([403, 100, 505, 414]), {'max_score': 0.0, 'lost': 0, 'score': 0.9937806725502014}), (87, 11, 15, array([325, 232]), array([290, 136, 360, 329]), {'max_score': 0.0, 'lost': 0, 'score': 0.8048837780952454}), (87, 12, 15, array([197, 252]), array([151, 115, 243, 390]), {'max_score': 0.0, 'lost': 0, 'score': 0.8022710084915161}), (87, 13, 16, array([552, 176]), array([494,  17, 610, 336]), {'max_score': 0.0, 'lost': 0, 'score': 0.7514221668243408})] 

Tracker Outputs:
 [(88, 0, 15, array([108, 258]), array([ 40,  89, 177, 427]), {'max_score': 0.0, 'lost': 0, 'score': 0.9866787791252136}), (88, 8, 15, array([452, 256]), array([401, 104, 503, 408]), {'max_score': 0.0, 'lost': 0, 'score': 0.9731889963150024}), (88, 11, 15, array([326, 231]), array([292, 132, 360, 331]), {'max_score': 0.0, 'lost': 0, 'score': 0.80928969

...

Tracker Outputs:
 [(777, 54, 16, array([588, 227]), array([546, 155, 631, 300]), {'max_score': 0.0, 'lost': 5, 'score': 0.279617577791214}), (777, 63, 15, array([341, 202]), array([302, 116, 381, 289]), {'max_score': 0.0, 'lost': 0, 'score': 0.44780856370925903}), (777, 65, 15, array([105, 243]), array([ -4,  11, 215, 476]), {'max_score': 0.0, 'lost': 0, 'score': 0.9984734654426575})] 

Tracker Outputs:
 [(778, 63, 15, array([326, 201]), array([281, 115, 371, 288]), {'max_score': 0.0, 'lost': 0, 'score': 0.5309173464775085}), (778, 65, 15, array([114, 242]), array([  0,   6, 228, 478]), {'max_score': 0.0, 'lost': 0, 'score': 0.9994945526123047})] 

Tracker Outputs:
 [(779, 63, 15, array([327, 204]), array([281, 120, 373, 288]), {'max_score': 0.0, 'lost': 0, 'score': 0.640249490737915}), (779, 65, 15, array([122, 242]), array([  8,   8, 236, 476]), {'max_score': 0.0, 'lost': 0, 'score': 0.9979876279830933})] 

Tracker Outputs:
 [(780, 63, 15, array([327, 205]), array([281, 123, 373, 288

Tracker Outputs:
 [(800, 63, 15, array([275, 244]), array([ 89,   8, 462, 480]), {'max_score': 0.0, 'lost': 0, 'score': 0.9943583607673645}), (800, 65, 15, array([171, 240]), array([  0,   0, 343, 480]), {'max_score': 0.0, 'lost': 5, 'score': 0.9746516942977905}), (800, 68, 15, array([299, 132]), array([219,   0, 379, 265]), {'max_score': 0.0, 'lost': 5, 'score': 0.954085648059845})] 

Tracker Outputs:
 [(801, 63, 15, array([287, 242]), array([115,   4, 460, 480]), {'max_score': 0.0, 'lost': 0, 'score': 0.997035026550293})] 

Tracker Outputs:
 [(802, 63, 15, array([311, 239]), array([147,   3, 475, 475]), {'max_score': 0.0, 'lost': 0, 'score': 0.9944181442260742}), (802, 70, 16, array([592, 218]), (552, 147, 633, 289), {'max_score': 0.0, 'lost': 0, 'score': 0.0})] 

Tracker Outputs:
 [(803, 63, 15, array([317, 236]), array([145,   0, 490, 473]), {'max_score': 0.0, 'lost': 0, 'score': 0.9950112104415894}), (803, 70, 16, array([592, 218]), (552, 147, 633, 289), {'max_score': 0.0, 'lost':

Tracker Outputs:
 [(831, 73, 15, array([326, 196]), array([283, 108, 369, 284]), {'max_score': 0.0, 'lost': 0, 'score': 0.47677674889564514}), (831, 74, 16, array([ 44, 215]), (2, 171, 87, 260), {'max_score': 0.0, 'lost': 0, 'score': 0.0})] 

Tracker Outputs:
 [(832, 73, 15, array([327, 197]), array([285, 110, 369, 284]), {'max_score': 0.0, 'lost': 0, 'score': 0.44471079111099243}), (832, 74, 16, array([ 44, 215]), (2, 171, 87, 260), {'max_score': 0.0, 'lost': 1, 'score': 0.0})] 

Tracker Outputs:
 [(833, 73, 15, array([343, 200]), array([300, 113, 386, 288]), {'max_score': 0.0, 'lost': 0, 'score': 0.5029913783073425}), (833, 74, 16, array([ 49, 218]), array([  6, 176,  93, 260]), {'max_score': 0.0, 'lost': 0, 'score': 0.2917478084564209})] 

Tracker Outputs:
 [(834, 73, 15, array([339, 203]), array([298, 115, 381, 292]), {'max_score': 0.0, 'lost': 0, 'score': 0.5421240925788879}), (834, 74, 16, array([ 49, 218]), array([  6, 176,  93, 260]), {'max_score': 0.0, 'lost': 1, 'score': 0.29

Tracker Outputs:
 [(866, 73, 15, array([327, 201]), array([292, 110, 362, 292]), {'max_score': 0.0, 'lost': 1, 'score': 0.41179612278938293}), (866, 74, 16, array([ 45, 217]), array([  6, 177,  85, 257]), {'max_score': 0.0, 'lost': 0, 'score': 0.2652619183063507})] 

Tracker Outputs:
 [(867, 73, 15, array([327, 201]), array([292, 110, 362, 292]), {'max_score': 0.0, 'lost': 2, 'score': 0.41179612278938293}), (867, 74, 16, array([558, 150]), array([499,   1, 618, 300]), {'max_score': 0.0, 'lost': 0, 'score': 0.3064402639865875})] 

Tracker Outputs:
 [(868, 73, 15, array([327, 201]), array([292, 110, 362, 292]), {'max_score': 0.0, 'lost': 3, 'score': 0.41179612278938293}), (868, 74, 16, array([560, 151]), array([499,   3, 622, 300]), {'max_score': 0.0, 'lost': 0, 'score': 0.26633790135383606})] 

Tracker Outputs:
 [(869, 73, 15, array([327, 201]), array([292, 110, 362, 292]), {'max_score': 0.0, 'lost': 4, 'score': 0.41179612278938293}), (869, 74, 16, array([554, 154]), array([484,   9, 62

Tracker Outputs:
 [(892, 73, 15, array([327, 193]), array([290, 102, 364, 284]), {'max_score': 0.0, 'lost': 0, 'score': 0.5344516634941101}), (892, 74, 16, array([560, 149]), array([492,   3, 629, 296]), {'max_score': 0.0, 'lost': 0, 'score': 0.258456289768219}), (892, 75, 16, array([ 41, 216]), array([  4, 174,  78, 259]), {'max_score': 0.0, 'lost': 0, 'score': 0.3277243971824646})] 

Tracker Outputs:
 [(893, 73, 15, array([324, 195]), array([285, 107, 364, 283]), {'max_score': 0.0, 'lost': 0, 'score': 0.4049881398677826}), (893, 74, 16, array([560, 149]), array([492,   3, 629, 296]), {'max_score': 0.0, 'lost': 1, 'score': 0.258456289768219}), (893, 75, 16, array([ 45, 216]), array([  6, 176,  85, 257]), {'max_score': 0.0, 'lost': 0, 'score': 0.27987754344940186})] 

Tracker Outputs:
 [(894, 73, 15, array([324, 197]), array([285, 112, 364, 283]), {'max_score': 0.0, 'lost': 0, 'score': 0.3607991933822632}), (894, 74, 16, array([560, 149]), array([492,   3, 629, 296]), {'max_score': 0.0